In [0]:
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os

In [2]:
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

In [0]:
df = pd.DataFrame({'label':dataset.target, 'text':dataset.data})

In [7]:
df.shape

(11314, 2)

In [0]:
df = df[df['label'].isin([1,10])]
df = df.reset_index(drop = True)

In [12]:
df['label'].value_counts()

10    600
1     584
Name: label, dtype: int64

In [0]:
df['text'] = df['text'].str.replace("[^a-zA-Z]", " ")

In [14]:
df

,label,text
0,10,Well I will have to change the scoring on my ...
1,1,Archive name graphics resources list part La...
2,10,And of course Mike Ramsey was at one time ...
3,10,As I promised I would give you the name of th...
4,10,GAME S OF ADIRONDACK C...
...,...,...
1179,10,The Hawks win Jermey Roenick scored his ...
1180,10,I think that NHLPA is the best video game ...
1181,1,I am in the market for a bit graphics card...
1182,1,Hi there is there anybody who know a polygon...


In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from nltk.corpus import stopwords 
stop_words = stopwords.words('english')

In [0]:
tokenized_doc = df['text'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
detokenized_doc = [] 
for i in range(len(df)): 
    t = ' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 

df['text'] = detokenized_doc

In [20]:
df

,label,text
0,10,Well I change scoring playoff pool Unfortunate...
1,1,Archive name graphics resources list part Last...
2,10,And course Mike Ramsey one time captain Buffal...
3,10,As I promised I would give name Panther presid...
4,10,GAME S OF ADIRONDACK CDI Adirondack leads seri...
...,...,...
1179,10,The Hawks win Jermey Roenick scored th goal Ha...
1180,10,I think NHLPA best video game available course...
1181,1,I market bit graphics card PC ISA bus wonderin...
1182,1,Hi anybody know polygon reduction algorithm ma...


In [0]:
from sklearn.model_selection import train_test_split
df_trn, df_val = train_test_split(df, stratify = df['label'], test_size = 0.4, random_state = 12)

In [22]:
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "")
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val, vocab=data_lm.train_ds.vocab, bs=32)

In [28]:
learn = language_model_learner(data_lm,  arch = AWD_LSTM, pretrained = True, drop_mult=0.7)

In [29]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,6.079857,5.173303,0.243575,06:20


In [0]:
learn.save_encoder('ft_enc')

In [33]:
learn = text_classifier_learner(data_clas, arch = AWD_LSTM, pretrained = True, drop_mult=0.7)
learn.load_encoder('ft_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (710 items)
x: TextList
xxbos xxmaj it looks like xxmaj edmonton xxmaj oilers decided take xxmaj european xxunk spring xxmaj ranford xxmaj tugnutt xxmaj benning xxmaj manson xxmaj smith xxmaj buchberger xxmaj corson playing xxmaj canada xxmaj podein xxmaj weight playing xxup us xxmaj is xxmaj kravchuk playing xxmaj xxunk i know nagging injuries late season xxmaj podein interesting case eligible play xxmaj cape xxmaj breton xxup ahl playoffs like xxmaj kovalev xxmaj zubov xxmaj andersson obviously xxmaj sather xxmaj pocklington total xxunk everyone makes certainly case massively xxunk xxmaj paramount xxmaj new xxmaj york xxmaj rangers,xxbos xxmaj this xxunk xxmaj speaking die hard i i read xxunk hard xxunk xxmaj toronto xxmaj cup finals xxmaj first anyone planet heard team xxmaj detroit xxmaj al xxmaj xxunk however spell idiot name must xxmaj chicago xxup espn said even close xxmaj chicago xxunk win xxmaj norris xxmaj division xxmaj p

In [34]:
learn.fit_one_cycle(1, 1e-2)


epoch,train_loss,valid_loss,accuracy,time
0,0.321961,0.227458,0.926160,17:47


In [35]:
preds, targets = learn.get_preds()

predictions = np.argmax(preds, axis = 1)
pd.crosstab(predictions, targets)

col_0,0,1
row_0,,
0,232,33
1,2,207
